In [3]:
import json

with open(r"C:\Users\ragur\Japanese_Learning_Application\Mnemonics\merged_kanji.json", "r", encoding="utf-8") as f:
    kanji_data = json.load(f)

print(len(kanji_data))


2230


In [4]:
import pandas as pd

radicals_df = pd.read_csv(r"C:\Users\ragur\Japanese_Learning_Application\Mnemonics\radicals_with_visual_form_20251029_104855.csv")
print(f"✅ Loaded {len(radicals_df)} radicals.")


✅ Loaded 322 radicals.


In [5]:
radical_docs = []

for _, row in radicals_df.iterrows():
    radical = str(row["Radical"])
    meaning = str(row["Meaning"])
    visual = str(row.get("Visual_Form", ""))
    text = f"Radical: {radical}\nMeaning: {meaning}\nVisual: {visual}"
    radical_docs.append(text)

print(f"✅ Prepared {len(radical_docs)} radical documents for embeddings.")


✅ Prepared 322 radical documents for embeddings.


In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
radical_embeddings = embedder.encode(radical_docs, show_progress_bar=True)

dimension = radical_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(radical_embeddings))

print("✅ Radical FAISS index built!")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

✅ Radical FAISS index built!


In [7]:
import faiss
import numpy as np

dimension = radical_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(radical_embeddings))

print("✅ FAISS index built!")

✅ FAISS index built!


In [8]:
def retrieve_relevant_radicals(query_text, top_k=3):
    query_emb = embedder.encode([query_text])
    D, I = index.search(np.array(query_emb), top_k)
    return [radical_docs[i] for i in I[0]]

In [9]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

def generate_mnemonic(kanji):
    k = kanji.strip()
    details = kanji_data.get(k)
    if not details:
        return f"Kanji {k} not found."

    meanings = ", ".join(details.get("meanings", []))
    radicals_en = details.get("wk_radicals", [])
    
    # Retrieve possible radical info from your radical CSV (as text snippets)
    radical_context = "\n\n".join(retrieve_relevant_radicals(" ".join(radicals_en)))

    # --- 🧠 Key change ---
    # We now *ask the model* to interpret and choose the correct radical symbols.
    prompt = f"""
You are a Japanese Kanji expert and mnemonic generator.
You create realistic, logical mnemonics for kanji using radical knowledge.

For each English radical name, infer the most appropriate Japanese radical symbol (部首)
based on the context and meaning of the kanji. 
Use the radical database info below as reference.

Radical Database:
{radical_context}

Now, generate a short mnemonic and reasoning in this table format:

| Kanji | Meaning | Radicals (Japanese Symbols) | Mnemonic | Reminder |
|-------|----------|-----------------------------|-----------|-----------|
| {k} | {meanings} | {', '.join(radicals_en)} |

Rules:
- Replace English radicals with the correct radical symbols.
- Use natural, realistic logic (e.g., 買 = Net + Shell → Buying involves covering valuable shells).
- Keep mnemonic short, 1–2 sentences.
"""

    result = pipe(prompt, max_new_tokens=150, temperature=0.25, top_p=0.8)
    return result[0]["generated_text"]


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [10]:
print(generate_mnemonic("買"))


You are a Japanese Kanji expert and mnemonic generator.
You create realistic, logical mnemonics for kanji using radical knowledge.

For each English radical name, infer the most appropriate Japanese radical symbol (部首)
based on the context and meaning of the kanji. 
Use the radical database info below as reference.

Radical Database:
Radical: ⼀
Meaning: one, horizontal stroke
Visual: Horizontal line.

Radical: ⼁
Meaning: vertical stroke
Visual: 竖笔直立

Radical: ⼂
Meaning: dot
Visual: Dot shape, often small and simple.

Now, generate a short mnemonic and reasoning in this table format:

| Kanji | Meaning | Radicals (Japanese Symbols) | Mnemonic | Reminder |
|-------|----------|-----------------------------|-----------|-----------|
| 買 | Buy | Net, Shell |

Rules:
- Replace English radicals with the correct radical symbols.
- Use natural, realistic logic (e.g., 買 = Net + Shell → Buying involves covering valuable shells).
- Keep mnemonic short, 1–2 sentences.
- Include reminder if applicab

In [11]:
print(generate_mnemonic("人"))


You are a Japanese Kanji expert and mnemonic generator.
You create realistic, logical mnemonics for kanji using radical knowledge.

For each English radical name, infer the most appropriate Japanese radical symbol (部首)
based on the context and meaning of the kanji. 
Use the radical database info below as reference.

Radical Database:
Radical: ⼀
Meaning: one, horizontal stroke
Visual: Horizontal line.

Radical: ⼁
Meaning: vertical stroke
Visual: 竖笔直立

Radical: ⼂
Meaning: dot
Visual: Dot shape, often small and simple.

Now, generate a short mnemonic and reasoning in this table format:

| Kanji | Meaning | Radicals (Japanese Symbols) | Mnemonic | Reminder |
|-------|----------|-----------------------------|-----------|-----------|
| 人 | Person | Person |

Rules:
- Replace English radicals with the correct radical symbols.
- Use natural, realistic logic (e.g., 買 = Net + Shell → Buying involves covering valuable shells).
- Keep mnemonic short, 1–2 sentences.
- Include a reminder to help re

In [12]:
print(generate_mnemonic("会"))


You are a Japanese Kanji expert and mnemonic generator.
You create realistic, logical mnemonics for kanji using radical knowledge.

For each English radical name, infer the most appropriate Japanese radical symbol (部首)
based on the context and meaning of the kanji. 
Use the radical database info below as reference.

Radical Database:
Radical: ⼀
Meaning: one, horizontal stroke
Visual: Horizontal line.

Radical: ⼁
Meaning: vertical stroke
Visual: 竖笔直立

Radical: ⼂
Meaning: dot
Visual: Dot shape, often small and simple.

Now, generate a short mnemonic and reasoning in this table format:

| Kanji | Meaning | Radicals (Japanese Symbols) | Mnemonic | Reminder |
|-------|----------|-----------------------------|-----------|-----------|
| 会 | Meeting, Meet, Party, Association, Interview, Join | Meet |

Rules:
- Replace English radicals with the correct radical symbols.
- Use natural, realistic logic (e.g., 買 = Net + Shell → Buying involves covering valuable shells).
- Keep mnemonic short, 1–2 

In [13]:
print(generate_mnemonic("川"))


You are a Japanese Kanji expert and mnemonic generator.
You create realistic, logical mnemonics for kanji using radical knowledge.

For each English radical name, infer the most appropriate Japanese radical symbol (部首)
based on the context and meaning of the kanji. 
Use the radical database info below as reference.

Radical Database:
Radical: ⼀
Meaning: one, horizontal stroke
Visual: Horizontal line.

Radical: ⼁
Meaning: vertical stroke
Visual: 竖笔直立

Radical: ⼂
Meaning: dot
Visual: Dot shape, often small and simple.

Now, generate a short mnemonic and reasoning in this table format:

| Kanji | Meaning | Radicals (Japanese Symbols) | Mnemonic | Reminder |
|-------|----------|-----------------------------|-----------|-----------|
| 川 | Stream, River, River Or Three-stroke River Radical (no. 47) | River |

Rules:
- Replace English radicals with the correct radical symbols.
- Use natural, realistic logic (e.g., 買 = Net + Shell → Buying involves covering valuable shells).
- Keep mnemonic s